In [2]:
from requests.exceptions import RequestException
from contextlib import closing
import re
from lxml import html
import requests
from datetime import datetime
import Algorithmia
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"./climate_sentiment_key.json"

In [3]:
client = Algorithmia.client('sim0ds3RRSQiux4Q9vb47cqyuHe1')
algo = client.algo('nlp/SentimentAnalysis/1.0.5')
def get_algorithmia_sentiment(text):
    algo_in = {
          "document": text
        }
    return algo.pipe(algo_in).result[0]['sentiment']

In [4]:
# Instantiates a client
client = language.LanguageServiceClient()

def get_gcp_sentiment(text):
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)
    # Detects the sentiment of the text
    sentiment = client.analyze_sentiment(document=document).document_sentiment
    return sentiment.score

In [5]:
def get_mean_sentiment(sentiments):
    return sum([x[0] for x in sentiments]) / len(sentiments)

In [42]:
url_data = {}
def get_url_data(url):
    if url not in url_data:
        base_url = "https://www.textise.net/showText.aspx?strURL="
        print("Downloading HTML page...")
        page = requests.get(base_url + url.replace(":", "%253A"))
        tree = html.fromstring(page.content)
        print("Finished downloading. Parsing...")

        divs = []
        delete = [" |", r"\n        Other"]
        for div in tree.xpath('//div/text()'):
            divs.append(div.rstrip())

        divs = divs[8:]
        divs = filter(lambda x: x != '', divs)
        divs = filter(lambda x: x not in delete, divs)
        divs = filter(lambda x: "[Image: " not in x, divs)
        date_re = r'(\w+ \d+, \d+)'
        divs = list(divs)
        divs = divs[:-10]
        divs = divs[3:]
        #date = datetime.strptime(divs[0], "Published %B %d, %Y")
        #divs = divs[2:]
        url_data[url] = divs
        return divs
    else:
        #print("Found URL data in cache.")
        return url_data[url]

In [43]:
def get_date_from_url(url):
    divs = get_url_data(url)
    date = datetime.strptime(divs[0], "Published %B %d, %Y")
    return date

In [44]:
def analyze_sentiment_by_sentences(url, verbose=False):
    divs = get_url_data(url)
    date = get_date_from_url(url)
    
    return(divs, date)

In [45]:
sentiments = analyze_sentiment_by_sentences("https://www.foxnews.com/transcript/weather-channel-founder-blasts-climate-change", verbose=True)

Finished downloading. Parsing...


In [46]:
sentiments

(['Published October 28, 2014',
  'Last Update January 27, 2017',
  'Weather Channel founder blasts climate change',
  "John Coleman calls trend 'fake'",
  'This is a rush transcript from "The Kelly File," October 27, 2014. This copy may not be in its final form and may be updated.',
  "MEGYN KELLY, HOST: The administration hammering the message that manmade climate change is one of mankind's greatest threats. But our next guest disagrees. John Coleman is the co- founder of the Weather Channel, winner of the American Meteorological -- that's a tough one -- Meteorological Society's award for broadcast meteorologist of the year and he has spent more than 60 years reporting on the weather.",
  'John, good to see you tonight. And so upset were you about the claims of manmade global warming that you wrote a letter to the UCLA\'s Hammer Museum form their forum that was called Tackling Climate Change. And you said, "You don\'t have both sides represented, and I\'m here to tell you that manmad